In [ ]:
// Importieren der benötigten Module
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE, 0.0.1"
#r "nuget: FSharp.Stats"

// ... und öffnen der benöätigten Module
open FsODE
open Plotly.NET
open FSharp.Stats


// der Rechenalgorithmus der Programmbibliothek, die wir benutzen:
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

Installed Packages FSharp.Stats, 0.4.7 FsODE, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

### Einfaches Model vom Lac Operon

Einleitender Text der das Lac Operon erklärt








Um die Komplexität des Lac Operons zu verringern wird sich im Folgenden nur die Konzentration von intrazellulärer Lactose angeschaut. Damit diese Betrachtung möglichst einfach bleibt wird davon ausgegangen das es genug extrazelluläre Lactose gibt. Lactose kann wie oben beschrieben nur durch $\beta -\textrm {Galactosid-Permease}$ in die Zelle transportiert werden. LacY, das Gen das $\beta -\textrm {Galactosid-Permease}$ codiert, wird kontinuierlich expremiert auch wenn es durch LacI inhibiert wird. Allerdings ist die produzierte Menge an $\beta -\textrm {Galactosid-Permease}$ sehr gering. Das heißt Lactose kann immer in die Zelle transporitert werden, aber nur in geringen Mengen solange LacI noch an den Promotor des Lac Operons bindet. <br>
Wenn die Konzentration von Lactose in der Zelle hoch genug ist, kommt es wie besprochen zur Bildung von Allolactose welches eine Bindung mit LacI eingeht. Dies führt dazu das LacI nicht mehr an den Promotor des Lac Operons bindet was zu einer erhöhten Produktion von  $\beta -\textrm {Galactosid-Permease}$ führt. Somit wird mehr Lactose in die Zelle transportiert. <br>
Die Differenzialgleichung für die Konzentration von intrazellulärer Lactose kann somit folgendermaßen beschrieben werden. Es gibt einen unregulierten geringen Lactoseimport, genannt $\alpha$. Dieser ist nicht reguliert. Es gibt außerdem einen regulierten Lactoseimport, dieser wird $\beta$ genannt analog zur Proteinproduktion von $\beta -\textrm {Galactosid-Permease}$ die nötig ist damit dieser regulierte Transport beginnt. Da mehr intrazelluläre Lactose zu mehr Import führt, kann hier von einer positiven Autoregulation gesprochen werden. Abschließend wird Lactose natürlich auch von der Zelle verbraucht, wobei die Menge die genutzt wird wieder von der Lactosekonzentration abhängt. Die Differenzialgleichung ist somit <br>

$$ Lactosekonzentration_{intrazellulär}' = \frac {dLactosekonzentration_{intrazellulär}} {dt}= \alpha + \beta * \frac {Lactosekonzentration_{intrazellulär}}{K_{D} + Lactosekonzentration_{intrazellulär}} - \gamma*Lactosekonzentration_{intrazellulär}$$
Da die Simulationen nur auf die richtige Differenzialgleichung achten, aber nicht auf die genutzten Namen der Parameter verkürzen wir die Gleichung zu
$$ inlac' = \frac {dinlac} {dt}= \alpha + \beta * \frac {inlac}{K_{D} + inlac} - \gamma*inlac$$


Aufgabe 4.1: <br>
Setzen sie die Gleichung für die intrazelluläre Lactosekonzentration in die nachfolgende Simulation ein (Bitte beachten sie, dass sie die Lactosekonzentration als "inlac" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\alpha$, $\beta \textrm{ und } \gamma$ als "alpha", "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration

        //Differenzialgleichung die diese Änderung beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha + beta * (inlac)/((inlac) + (K_d)) - (gamma*inlac)

        [| inlac' |]

//Startkonzentrationen
let P0 = [| 
    0.1
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = Ps |> SolPoints.toPoints 1 
    lactose |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Lactosekonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Aufgabe 4.2: <br>
Varrieren sie die Startkonzentration. Was beobachten sie? Ist dies das Verhalten was sie in einer Zelle erwarten würden? Begründen sie die Antwort. <br>
Kopieren sie dafür den obenstehnden Code Block in den folgenden Code Block und ändern sie dann die Werte für die Startkonzentration im Abschnitt Startkonzentrationen, der Wert unter P0 (lassen sie alles andere gleich).

In [ ]:
//hier code einfügen

Bitte hier die Antwort eingeben

Die Beobachtung zeigt das die aktuelle Differnzialgleichung nicht das darstellt was wir eigentlich zeigen wollen. Daher nehmen wir eine kleine Änderung am aktuellen Modell vor um den erwünschten Effekt zu erzielen. Die neue Differnzialgleichung lautet
$$ inlac' = \frac {dinlac} {dt}= \alpha + \beta * \frac {inlac^{2}}{(K_{D})^{2} + inlac^{2}} - \gamma*inlac$$

Aufgabe 4.3: <br>
Setzen sie die Gleichung für intrazelluläre Lactosekonzentration, in der der Teil der postiven Autoregulation hoch 2 genommen wird, in die nachfolgende Simulation ein (Bitte beachten sie, dass sie die Lactosekonzentration als "inlac" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\alpha$, $\beta \textrm{ und } \gamma$ als "alpha", "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration

        //Differenzialgleichung die diese Änderung beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha + beta * (inlac**2.)/((inlac**2.) + (K_d**2.)) - (gamma*inlac)

        [| inlac' |]

//Startkonzentrationen
let P0 = [| 
    0.1
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = Ps |> SolPoints.toPoints 1 
    lactose |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Lactosekonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Aufgabe 4.4: <br>
Was beobachten sie?<br>




Hier die Antwort eingeben

Aufgabe 4.5: <br>
Erhöhen sie jetzt Schrittweise den Wert von der $Lactosekonzentration(0)$. Kopieren sie dafür den obenstehnden Code Block in den folgenden Code Block und ändern sie dann die Werte für die Startkonzentration im Abschnitt Startkonzentrationen, der Wert unter P0 (lassen sie alles andere gleich). Nutzen sie dabei die folgenden Werte:
$$0.5$$
$$1.0$$
$$1.5$$
$$2.0$$
$$2.5$$
$$3.0$$
$$3.5$$
$$4.0$$
$$5.0$$
$$6.0$$
$$7.0$$
$$8.0$$

Notieren sie die Werte die sie für die Gleichgewichtspunkte finden, bei jeder Konzentration (der Wert bei dem sich die Konzentration nicht mehr merklich ändert)


Was können sie im Bezug auf das $Lactosekonzentration_{GGW}$ sagen?


Block für die Werte und die Antwort

Hier die Werte für die einzelnen $Lactosekonzentration_{GGW}$ eintragen:




Anschliesend bitte beantworten was sie über das  $Lactosekonzentration_{GGW}$ im allgemeinen sagen können.

Es folgt eine kurze Besprechung. 
### Expertenaufgabe?

----------------
Aufgabe 4.6: <br>
Bestimmen Sie, ob es sich um stabile oder instabile Punktattraktoren handelt. Nutzen sie dafür die Werte die sie oben eingetragen haben.
Stabil bedeutet das die $Lactosekonzentration{GGW}$ vom folgenden und vorherigen $Lactosekonzentration(0)$ identisch sind. Als Beispiel (diese Werte sind ausgedacht und sollen nur als Beispiel dienen):
$Lactosekonzentration(0)$ wird zu $Proteinkonzentration_{GGW}$
$$Lactosekonzentration(1.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(2.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(3.0) -> Lactosekonzentration{GGW} = 2$$

dann handelt es sich beim Punktattraktor 2 um einen stabilen Punktattraktor. 

Instabil bedeutet analog das die beiden flannkierenden Proteinkonzentration_{GGW} zu einem anderen Punktattraktor wandern. Als Beispiel:

$$Lactosekonzentration(3.0) -> Lactosekonzentration{GGW} = 2$$
$$Lactosekonzentration(4.0) -> Lactosekonzentration{GGW} = 4$$
$$Lactosekonzentration(5.0) -> Lactosekonzentration{GGW} = 8$$

Hier wäre der Punktattraktor bei 4 instabil. 

Gehen sie jezt bei denen von ihnen bestimmten Punkten anlog vor (es reicht wenn sie schreiben Punktattraktor X ist stabil/instabil).

Hier die Antwort eingeben.

Ein System das zwei stabile Punktattraktoren hat wird auch bistabiles System genannt. **Bistabilität** ist eine Schlüsseleigenschaften von biologischen Systemen, da das System auch dann an einem der beiden Punktattraktoren verweilt, wenn es leichte Störungen oder es leicht veränderte äußere Einflüsse gibt. Das bedeutet bezogen auf das lac Operon, das dieses nur dann exprimiert wird wenn es eine hohe Konzentration an extrazellulärer Lactose gibt. Bei nur sehr geringen Mengen kommt es nicht zur Ausbildung von Allolactose und das lac Operon wird weiterhin durch lacI inhibiert. Dies hat den Vorteil das die Energie die auf die Produktion von lacY, laxZ, und lacA verwendet werden würde gespart wird, da die Menge an Lactose dies nicht rechtfertigt. Auf der anderen Seite kommt es bei einer hohen Lactosekonzentration zur Ausbildung von genug Allolactose, so dass das Lac Operon nicht mehr durch LacI inhibiert wird. Da die Repression sehr stark ist, der Faktor ist ungefähr 1000, kommt es anschließend zu einer sehr starken Expression des Lac Operon. <br>
Die Fähigkeit nur dann Gene zu exprimieren, wenn sie wirklich gebraucht werden, aber dann in sehr hohem Maße, ist ein grundlegender Vorteil gegenüber einem Organismus der diese Fähigkeit nicht hat und immer alles mit der selben Rate produziert. Daher hat sich **Bistabilität** als eines der Grundlegenden Bausteine in der Evolution durchgesetzt und kann heute in jedem Organismus bei fast allen relevanten Genzyklen beobachtet werden.
Bistabilität ist allerdings nur eine spezialform der **Multistabilität**. Multistabilität bedeutet das ein System mehrere Fixpunkte hat zu denen es wandern kann, anhängig von den äußeren Einflüssen. Dies hat beim System von Lactose noch keinen Vorteil, aber bei komplexeren Problemen wo wie der Proliferation ist es deutlich vorteilhafter. 



Aufgabe 4.7: <br>
Teilen sie Gleichung welche die Änderung der Lactosekonzentration im Bezug auf die Zeit beschreibt in zwei Gleichungen. Die erste soll die Zunahme der Lactose beschreiben, die zweite die Abnahame, also
$$ \frac {dLactosekonzentration} {dt} = \textrm{Zunahme der Lactosekonzentration} $$
$$ \frac {dLactosekonzentration} {dt} = \textrm{Verringerung der Lactosekonzentration}$$

Fügen sie die beiden Gleichungen in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{d}$ als K_d eingesetzt wird). 

In [ ]:
//Konstanten
let alpha =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let K_d =    5.0  // Dissoziationskonstante

//Graphische Darstellung der Gleichungen
[
    // Fügen sie die Gleichung für den Proteinabbau ein

    Chart.Line [ for inlac in 0.0 .. 0.1 .. 15.0 do
    (inlac, (gamma*inlac)) ]
    |> Chart.withTraceInfo("Zunahme");
    //Fügen sie die Gleichung für die Proteinproduktion ein
    Chart.Line [ for inlac in 0.0 .. 0.1 .. 15.0 do 
    (inlac, alpha + beta * (inlac**2.)/((inlac**2.) + (K_d**2.))) ]
    |> Chart.withTraceInfo("Verringerung");
]

|> Chart.combine
|> Chart.withXAxisStyle("Lactosekonzentration")
|> Chart.withYAxisStyle("Veränderung der Lactosekonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Der gezeigte Graph stellt auf der Y-Achse die Änderung der Lactosekonzentration dar, also Import' und Abbau'. Die X-Achse repräsentiert die Lactosekonzentration. Diese graphische Darstellung macht es sehr einfach die Punktattraktionen zu bestimmen. Diese sind die Schnittpunkte der beiden Kurven, da bei diesen der Import und der Verbrauch den gleichen Wert besitzen. In diesem Fall befinden sich die Schnittpunkte der beiden Kurven bei 1, 2.5 und 7.3 .<br>
Diese Art von Graph ermöglicht es auch zu bestimmen ob es sich um stabile oder instabile Punktattraktionen handelt. Dafür muss man sich nur überlegen, wohin die Lactosekonzentration wandert. <br>
Einige Beispiele folgen: <br>
$$Lactosekonzentration (0) = 1$$ 
Dieser Wert liegt bereits auf einer Punktattraktion und daher wird die Lactosekonzentration sich nicht ändern.
$$Lactosekonzentration (0) = 2$$ 
Bei dieser Lactosekonzentration ist der Wert von der Kurve für die Verringerung höher als der für den Import. Das heißt es wird mehr Lactose verbraucht als importiert, somit sinkt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 1.
$$Lactosekonzentration (0) = 2.5$$
Dieser Wert liegt bereits auf einer Punktattraktion und daher wird die Lactosekonzentration sich nicht ändern. 
$$Lactosekonzentration (0) = 3$$
Bei dieser Lactosekonzentration ist der Wert von der Kurve für den Import höher als der für den Verbrauch. Das heißt es wird mehr Lactose importiert als verbraucht, somit steigt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 7,3.
$$Lactosekonzentration (0) = 10$$
Bei dieser Lactosekonzentration ist der Wert von der Kurve für den Verbrauch höher als der für den Import. Das heißt es  wird mehr Lactose verbraucht als importiert, somit sinkt die Lactosekonzentration und die $Lactosekonzentration(t)$ wandert zu der Punktattraktion 7.3.

Das heißt die Punktattraktion 1 ist stabil, da höhere Werte sich zu diesem Wert hinbewegen (bis die nächste Punktattraktion erreicht wird).
Die Punktattraktion 2.5 ist instabil, da höhere Werte zur nächsten Punktattraktion, 7.3, wandern und niedrigere zur vorherigen Punktattraktion, 0. Nur wenn die $Lactosekonzentration(0) = 2.5$ kommt es zu diesem Punkt.

Aufgabe 4.8: <br>
Kopieren sie den obenstehenden Code Block und fügen sie diesen in den folgenden Code Block ein. 


In [ ]:
//Code block einfügen

Varrieren sie die Werte für die Abbaukonsante $\gamma$. Bestimmen sie die Punktattraktionen und ob diese stabil oder instabil sind. Notieren sie die Werte im nachfolgennden Block. Varrieren sie $\gamma$ von 0.7 bis 1.5 in 0.1 schritten (also 0.7,0.8.0.9,...).

Notieren sie hier die Werte


Aufgabe 4.9: <br>
Setzen sie jetzt die Werte in den nachfolgenden Code Block ein und führen sie diesen anschliesend aus.
Beachten sie dabei das folgende: <br>
x Werte sind die Werte die sie für die Abbaukonstante $\gamma$ haben, y Werte sind die von ihnen notierten Punktattraktionen. 
Beachten sie außerdem das sie die Werte sinnvoll anordnen müßen um anschliesend ein aussagekräftiges Diagramm zu erhalten (Tipp: Der Kurvenverlauf folget der Ordnung in der sie Werte eingeben haben).

In [ ]:
let x = [0.7; 0.8; 0.9; 1.0; 1.1;0.9; 1.0; 1.1; 0.9; 1.0; 1.1; 1.2]
let y = [7. ; 7. ; 7. ;7. ; 7.  ; 2.5; 2.5; 2.5 ; 1. ; 1.; 1. ; 1.  ]

Chart.Spline (x, y)



<!-- Plotly chart will be drawn inside this DIV -->

Dieser Graph ist ein sogenanntes **Bifurkationsdiagramm**. **Bifurkation** beschreibt die qualitative Änderung des Zustandes eines Systems, das durch nicht lineare Gleichungen beschrieben wird, in Abhängigkeit der Veränderung eines Parameters der Teil der beschreibenden Gleichung ist. In diesem Fall haben sie den Parameter $\gamma$ verändert und es kam zu einer sogenannten **Saddle-Node Bifurkation**. Bei dieser Bifurkation verliert ein System zwei Punktattraktionen wenn der Bifurkationsparameter einen bestimmten Wert erreicht. Dabei wandern die beiden Punktattraktionen aufeinander zu, wenn sich der Bifurkationsparameter diesen Wert nähert und voneinander weg wenn er sich von diesem Wert entfernt. Das heißt das System wechselt zwischen den beiden Konformationen **bistabil** und **monostabil**, weshalb von einer qualitativen Änderung gesprochen wird.

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen.<br>

Expertenaufgabe 4.1: 

Stellen sie die Differenzialgleichung für ein Model auf das neben der intrazellulären Lactosekonzentration auch die Konzentration von $\beta -\textrm {Galactosid-Permease}$ umfasst.<br>
Fügen sie die Gleichungen in die nachfolgende Simulation ein (beachten sie das die Gleichung für die Lactosekonzentration die Konstanten "alpha1", "beta1", und "gamma1", ohne "", hat und die Gleichung für die Konzentration von $\beta -\textrm {Galactosid-Permease}$ die Konstanten "alpha2", "beta2", und "gamma2", ohne"". Die Konzentration von $\beta -\textrm {Galactosid-Permease}$ soll "perm", ohne "", genannt werden. ).

In [ ]:
//Konstanten
let alpha1 =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta1  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma1 =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let alpha2 =  0.5  // Konstante für die Zunahme der beta-Galactosid-Permease Konzentration (unreguliert)
let beta2  =  10.0 // Konstante für die Zunahme der beta-Galactosid-Permease Konzentration (reguliert)
let gamma2 =  0.5  // Konstante für die Verringerung der beta-Galactosid-Permease Konzentration
let K_d1   =  5.0  // Dissoziationskonstante1
let K_d2   =  5.0  // Dissoziationskonstante1

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration
        let perm  = P[1] // Änderung der beta-Galactosid Permease Konzentration

        //Differenzialgleichung die die Änderung in der Lactosekonzentration beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha1 + beta1 * (perm**2.)/((perm**2.) + (K_d**2.)) - (gamma1*inlac)
        //Differenzialgleichung die die Änderung in der beta-Galactosid Permease Konzentration beschreibt (hier die Gleichung einsetzen)
        let perm' = alpha2 + beta2* (inlac**2.)/((inlac**2.) + (K_d2**2.)) - (gamma2*perm)

        [| inlac'; perm' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 15 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = Ps |> SolPoints.toPoints 1 
    lactose |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
    let permease = Ps |> SolPoints.toPoints 2 
    permease |> Chart.Spline 
    |> Chart.withTraceInfo("Permease");

]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Konzentration")

<!-- Plotly chart will be drawn inside this DIV -->

---------
### Hill Funktion und Ultrasensitivität

Die Umgebung von einer Zelle ändert sich dauerhaft, da sich die Konzentrationen von allen Molekülen um die Zelle dauerhaft ändern. Durch aktiven oder passiven Import der Zelle, oder benachbarter Zellen, durch Export von bestimmten Molekülen, oder durch Diffusion und einige Möglichkeiten zu nennen. Die Zelle muss konstant die Expression von Genen und damit auch die Produktion von Proteinen an diese Veränderungen anpassen. Wie beim lac Operon gezeigt ist dies in vielen Fällen nicht möglich mit den am Vortrag besprochenen Differenzialgleichungen. Dies liegt daran das diesen Prozessen kein linearer Zusammenhang zu Grund liegt, sondern ein **ultrasensitives** Verhalten.
Das heißt das ein Prozess nur dann stattfindet wenn ein bestimmter Schwellenwert überschritten wird, während es davor keinen Effekt gibt. Oft ist dieses ultrasensitive Verhalten bedingt durch **Kooperativität**. Beim lac Operon führte dieses ultrasensitves Verhalten dazu das es nur bei hohen Anfangskonzentration zu einem Verstärkten Import von Lactose kam. Allerdings musste dies bewerkstelligt werden durch das Einführen eines Faktors hoch 2. Dieser Faktor beschreibt allerdings nicht alle möglichen Prozesse und ist auch nicht sehr allgemein. Daher wird für die mathematische Beschreibung von solchen Ultrasensitiven Prozessen in der Biologie zumeist die Hillfunktion verwendet. Das Anwenden von dieser auf die bisher besprochenen negative Autoregulation führt zu der folgenden Differenzialgleichung 
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ (\frac {Proteinkonzentration}{K_{D}})^{n}}  - \gamma*Proteinkonzentration$$

die Gleichung für eine ultrasentives Verhalten von einer Proteinproduktion die durch einen Repressor beeinflusst wird ist identisch, nur müssen sie die Proteinkonzentration im Term, der $\beta$ beeinflusst, durch die Repressorkonzentration ersetzten. Die Differenzialgleichung für positive Autoregulation ist 
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {Proteinkonzentration^{n}}{{(K_{D})^{n}} + Proteinkonzentration^{n}} - \gamma*Proteinkonzentration$$
auch hier führt das ersetzten der Proteinkonzentration, im $\beta$ beeinflussenden Term, mit der Aktivatorkonzentration zu der Gleichung für eine Proteinproduktion die von diesem beeinflusst wird. 
Der Parameter $n$ ist der Hillkoeffizent und beschreibt wie stark das ultrasensitve Verhalten ist. Ein Wert von 1 beschreibt die bisher besprochenen Verhaltensweisen, während ein Wert von 2 oder höher das ulatrsensitive Verhalten beschreibt. 
Im nachfolgenden soll überprüft werden, ob die Hill Funktion wirklich so wirkt wie besprochen.

Aufgabe 4.10:

Setzen sie die Gleichung für positive Autoregulation in die nachfolgende Simulation ein, dabei sollte die positive Autoregulation durch eine Hill Funktion beschrieben werden (Bitte beachten sie, dass sie die Proteinkonzentration als "Proteinkonznetration2" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).


In [ ]:
//Konstanten
let beta  =  10.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =    4.0  // Dissoziationskonstante
let n =      2.0  // Hill Koeffizent 

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonznetration1 = P[0] // Änderung in der Proteinmenge
        let Proteinkonznetration2 = P[1] // Änderung in der Proteinmenge
        //Änderung der Proteinkonzentration ohne positive Autoregulation
        let Proteinkonznetration1' = beta - (gamma*Proteinkonznetration1)
        // Änderung des Proteinkonzentration bei positiver Autoregulation (hier die Gleichung einsetzen)
        let Proteinkonznetration2' = beta * (Proteinkonznetration2**n)/((Proteinkonznetration2**n) + (K_d**n)) - (gamma*Proteinkonznetration2)

        [| Proteinkonznetration1'; Proteinkonznetration2' |]

//Startkonzentrationen
let P0 = [| 
    //Startkonzentration für das unregulierte Protein (muss nicht verändert werden)
    0.0 
    //Startkonzentration für das durch postive Autoreguliation regulierte Protein (hier bitte die Startwerte ändern)
    4.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("negative AR");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Die Konstanten für den gezeigten Graphen sind:
$$\beta = 10$$
$$\gamma= 1$$
$$n = 4$$
$$K_{D} = 1$$
$$Proteinkonzentration (0) = 0.1$$
Aufgabe 4.11: <br>
Welchen Effekt hat postive Autoregulation bei einem Wert von $n=1$ auf die Antwortszeit?


Hier die Antwort eintippen

Aufgabe 4.12 <br>
Was passiert bei $n=2$?
Kopieren sie die obenstehnden code block für die postive Autoregulation und fügen sie diesen in den folgenden Code Block ein. Verändern sie dann die Hill Konstante. 

Was beobachten sie?

Hier die Antwort eintippen

Es folgt eine kurze Besprechung.Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen.<br>

Expertenaufgabe 4.2: 

Stellen sie die Differenzialgleichung für ein Model auf das neben der intrazellulären Lactosekonzentration auch die Konzentration von $\beta -\textrm {Galactosid-Permease}$ umfasst auf, diesesmal soll der Zusammenhang aber mit der Hill Funktion gezeigt werden.
Fügen sie die Gleichungen in die nachfolgende Simulation ein (beachten sie das die Gleichung für die Lactosekonzentration die Konstanten "n1","alpha1", "beta1", und "gamma1", ohne "", hat und die Gleichung für die Konzentration von $\beta -\textrm {Galactosid-Permease}$ die Konstanten "n2", "alpha2", "beta2", und "gamma2", ohne"". Die Konzentration von $\beta -\textrm {Galactosid-Permease}$ soll "perm", ohne "", genannt werden. ).

In [ ]:
//Konstanten
let alpha1 =  0.5  // Konstante für die Zunahme der Lactosekonzentration (unreguliert)
let beta1  =  10.0 // Konstante für die Zunahme der Lactosekonzentration (reguliert)
let gamma1 =  1.0  // Konstante für die Verringerung der Lactosekonzentration
let alpha2 =  0.5  // Konstante für die Zunahme der beta-Galactosid-Permease Konzentration (unreguliert)
let beta2  =  10.0 // Konstante für die Zunahme der beta-Galactosid-Permease Konzentration (reguliert)
let gamma2 =  0.5  // Konstante für die Verringerung der beta-Galactosid-Permease Konzentration
let K_d1   =  5.0  // Dissoziationskonstante1
let K_d2   =  5.0  // Dissoziationskonstante1
let n1     =  3.0  // Hill Koeffizent für die Regulation der Lactosekonzentration
let n2     =  3.0  // Hill Koeffizent für die Regulation der beta-Galactosid Permease Konzentration

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let inlac = P[0] // Änderung in der Lactosekonzentration
        let perm  = P[1] // Änderung der beta-Galactosid Permease Konzentration

        //Differenzialgleichung die die Änderung in der Lactosekonzentration beschreibt (hier die Gleichung einsetzen)
        let inlac' = alpha1 + beta1 * (perm**n1)/((perm**n1) + (K_d**n1)) - (gamma1*inlac)
        //Differenzialgleichung die die Änderung in der beta-Galactosid Permease Konzentration beschreibt (hier die Gleichung einsetzen)
        let perm' = alpha2 + beta2* (inlac**n2)/((inlac**n2) + (K_d2**n2)) - (gamma2*perm)

        [| inlac'; perm' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 15 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let lactose = Ps |> SolPoints.toPoints 1 
    lactose |> Chart.Spline 
    |> Chart.withTraceInfo("Lactose");
    let permease = Ps |> SolPoints.toPoints 2 
    permease |> Chart.Spline 
    |> Chart.withTraceInfo("Permease");

]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Konzentration")

<!-- Plotly chart will be drawn inside this DIV -->

------------------

### Osszillation/Hopf Bifukration folgt

### Eine einfache biologische Uhr

Wir als lebende Organismen werden von vielen verschiedenen Uhren kontrolliert. Die uns bekannteste ist der zirkadiane Rhythmus. Dieser beschreibt die Fähigkeit physiologische Vorgänge, Schlafen, Nahrungsaufnahme, AKtivitäten, etc., auf eine periodenlänge von circa 24 Stunden zu synchronisieren. Wir nennen den zirkadianen Rhythmus auch umgangssprachlicher unseren Tagesrhythmus. <br>
Der Tagesrhythmus spiegelt direkt eine der wichtigsten Eigenschaften von biologischen Uhren wieder. Sie können sich von Störrungen erholen. Eine Nacht nicht zu schlafen, oder einen Tag nichts zu essen, führt im Normalfall nicht dazu das sich unser Tagesrhythmus ändert. Nach ein bis zwei Tagen ist alles wieder so wie davor. Nur das konstante Verändern der Lebenumstände, also zum Beispiel immer Nachts zu arbeiten, führt dazu das sich unser Tagesrhythmus umstellt. <br>
Periodische Zyklen , auch Oszillationen genannt, die auch bei kleinen Störrungen wieder in ihren ursprunglichen Zyklus zurückkehren werden Grenzzklus, oder Limit Cycle, genannt. Das folgende Bild veranschaulicht dies

Bild 1

Das Nachbilden eines so komplexen Rhythmuses wie den zirkadianen Rhythmus ist für dieses Praktikum leider zu komplex. Stattdessen wird im folgenden Teil eine einfache biologische Uhr aus Proteinen gebaut. Dabei wird zuerst das allgemeine Design der Uhr besprochen und darauffolgend gezeigt das dieses Design funktioniert, sowohl in silico als auch in vitro.
Das bauen einer einfachen biologischen Uhr mit den bereits gelernten Bausteinen, also eine Uhr die auf einem System von Genen und Proteinen beruht, muss auf der Idee basieren das die einzelnen Bausteine einen Grenzzyklus formen, das es ansonsten nicht zur Oszillation kommt. Weiterhin sollte diese einfache Uhr die Fähigkeit haben bei vielen verschiedenen biologischen Parametern zu oszillieren. Dies ist nötg da es sehr schwierig ist die biologischen Parameter in der Zelle zu kontrollieren. Im folgenden wird sich zuerst mit dem Design des Grenzzyklus beschäftigt und erst nachdem nachgewiesen ist das dieser so funktioniert wie gewünscht wird sich damit beschäftigt welche Proteine sinnvolle Kandiaten für dieses Design sind. <br>
Eine mögliches Desgin  für einen Grenzzyklus  ist ein "Stein-Schwere-Papier" Feedback loop. Das heißt drei Proteine die in einem Kreis angeordnet sind und sich gegenseitig inhibieren. Dieses Design wird allgemein auch Repressilator genannt.  Das folgende Diagramm veranschaulicht dieses Idee

Bild 2


Aus dem Schaubild geht hervor das sich die Differenzialgleichungen, welche die Proteinkonzentrationen der einzelnen Proteine beschreiben, miteinander verknüpft sind. Dies bedeutet die Differnzialgleichung für das erste Protein $x_{1}$ kann folgendermaßen hergleitet werden. Die Proteinproduktion $\beta$ wird reguliert durch den Repressor $x_{3}$. Die Regulierung selbst wird beschrieben durch eine Hillfunktion, da wir noch nicht wissen ob das Verhalten ultrasensitiv oder linear sein muss um die Oszillation zu erhalten. Die Verringerung, beschrieben durch $\gamma$ des Proteins $x_{1}$ ist nur von $x_1$ abhängig. Somit ergibt sich
$$x_{1}' = \frac {dx_{1}}{dt} = \beta * \frac {1} {1+ (\frac {x_{3}}{K_{D}})^{n}} - \gamma * x_{1}$$

Die Gleichungen für die anderen zwei Proteine, $x_{2}$ und $x_{3}$, werden analog aufgestellt.

Aufgabe 4.13: <br>
Stelle die beiden anderen Gleichungen auf

Um zu überprüfen ob das besprochene Model wirklich zu einem Grenzzyklus führt, wird im ersten Schritt ermittelt wie viele Punktattraktionen es gibt. Dafür wird das oben besprochene Model der Seperierung von Zunahme und Verringerung genutzt. Die Zunahme lässt sich somit beschreiben als 
$$ \beta * \frac {1} {1+ (\frac {x_{i+1}}{K_{D}})^{n}}$$
und die Verringerung als 
$$ \gamma * x_{i}  $$
wobei $i$ für das Protein steht das gerade betrachtet wird und $i+1$ das darauffolgende Protein beschreibt.
Da wir es hier mit drei Funktionen zu tun haben, die miteinander Verknüpft sind, ist für die Bestimmung der Punktattraktion also dem Punkt bei dem alle $x_{i}'=0$ sind nicht so einfach wie zuvor, da die gezeigten Zunahmen und Verringerungen erst zusammengebracht werden müssen. Die Lösung dieses Problems ist
$$x_{i} = fff(x_{i})$$
Da alle drei Funktionen identisch sind bis auf den Parameter $x_{i}$ kann der Zusammenhang als eine Kompositiion auf Funktionen beschrieben werden. Dies ermöglicht eien sehr einfache Berechnung der Differenzialgleichung, da einfach die Kettenregel auf jede Funktion angewendet werden kann. Die graphische Darstellung der Zunahme und Verringerung ist somit

In [ ]:
// Parameter
let beta, n = 3., 1.

// f(x)
let f x = beta / (1. + x**n)

// Herstellung der Komposition von Funktionen
let xx = [|0. .. 0.01 .. 3.|]
let fff = xx |> Array.map (fun x -> f(f(f x)))

[
    Chart.Line (xx,xx)
    |> Chart.withTraceInfo("Verringerung");
    Chart.Line (xx,fff)
    |> Chart.withTraceInfo("Zunahme ");
]
|> Chart.combine
|> Chart.withXAxisStyle("Proteinkonzentration")
|> Chart.withYAxisStyle("Veränderung der Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Es zeigt sich das dieses Model nur einen Punktattraktor hat. Um zu bestimmen welche Art der Punktattraktor ist, also stabil, instabil, etwas anderes, ist es leider nicht möglich das hier gezeigte Diagramm zu nutzen, da dieses durch eine Komposition von Funktionen enstanden ist. Daher müssen andere Methoden verwendet werden um die Punktattraktion zu finden und zu bestimmen um welche Art es sich handelt. Eines der meist genutzten Verfahren dafür ist Lineare Stabilität Analyse. Diese auf das hier gezeigte Problem anzuwenden führt zu folgender Lösung:
1. Die Punktattraktion befindet sich an dem Punkt der die folgende Gleichung erfüllt

$$ \beta= x_{0}* (1+ (x_{0})^{n})$$
wobei $x_{0}$ die Punktattraktion beschreibt

2. Es handelt sich um einen Grenzzyklus solange n>2 ist
3. Bei einem Wert von n<2 gibt es keine Oszillation

Diese Ergebnisse sagen uns das das gewählte Desgin zu einem Grenzzyklus führt, solange n>2 ist.
Um nun den nummerischen Wert für die Punktattraktion zu finden, muss die Gleichung $ \beta= x_{0}* (1+ (x_{0})^{n})$ gelöst werden in Abhängigkeit von $\beta$. Dies ist analystisch nur äußerst schwierig zu machen, aber es gibt Algorithmen die dieses für uns machen. Ein solcher ist der Bisection Algortihmus. Anwendung von diesem bringt uns zu folgendem Graphen

In [ ]:
//Bisection Algorithmus (nicht anfassen)
let fixedPoint beta (n: float) =
    Optimization.Bisection.tryFindRoot
        (fun x -> beta - x * (1. + x**n))
        0.0001
        0.
        beta
        100
//Array der beschreibt welche Spanne von betas sich angeschaut werden (von 0 bis 100) und mit welchen Schritten (0.5)
let betaArr = [|0. .. 0.5 .. 100.|]

//Die gewählten Werte für n (diese Verändern falls nötig)
[|2.;4.;8.|]
//Die graphische Darstellung der Fixpunkte, berechnet durch den Bisection Algorithmus (nicht anfassen)
|> Array.map (fun n ->
    Chart.Line
        (
            Array.zip 
                betaArr
                (betaArr |> Array.map (fun beta -> fixedPoint beta n))
            |> Array.filter (fun (x,y) -> y.IsSome)
            |> Array.map (fun (x,y) -> x,y.Value)
        )
    |> Chart.withTraceInfo(Name = $"n={n} ");

)
|> Chart.combine
|> Chart.withXAxisStyle("Beta")
|> Chart.withYAxisStyle("Punktattraktion")

<!-- Plotly chart will be drawn inside this DIV -->

Das bestimmen des nummerischen Wertes auf diesem Graph ist trivial. Wir müssen uns nur überlegen welches $\beta$ wir haben und welches $n$ und können dann einfach den Wert an der gegeben Stelle ablesen.

Aufgabe 4.14:
Ermittle die Punktattraktion für die folgenden drei Kombinationen von $\beta$ und $n$. 
1. $\beta = 10$, $n=3$
2. $\beta = 10$, $n=10$
3. $\beta = 2$ , $n=10$

Kompiere dafür den Code Block mit dem Bisektion Algorithmus in den nachfolgenden Code Block und gebe verändere die Werte für n, falls nötig.

In [ ]:
//Hier den Code reinkopieren

Hier die Antwort eintragen

Es folgt eine kurze Besprechung. Falls sie zu schnell vorangekommen sind können sie die folgende Aufgabe bearbeiten.

Expertenaufgabe 4.3: 

----------------------

Wir werden jetzt dazu übergehen den Grenzzyklus graphisch darzustellen. 

Aufgabe 4.15:<br>
Fügen sie die drei Differenzialgleichungen die die Prteinkonzentration von x1, x2, und x3 beschreiben in den nachfolgenden Code Block ein (beachten sie das übliche).

In [ ]:
//Konstanten
let beta  = 5. // Konstante für die Zunahme der Proteinkonzentration
let gamma = 1.  // Konstante für die Abnahme der Proteinkonzentration 
let K_d   = 1.  // Dissoziationskonstante
let n     = 3.  // Hillkonstante

let repressilator_rhs (x1,x2,x3) t beta n =
    [|
        beta / (1. + x3 ** n) - x1
        beta / (1. + x1 ** n) - x2
        beta / (1. + x2 ** n) - x3
    |]

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->
        let x1 = P[0] //Änderung der Proteinkonzentration von x1
        let x2 = P[1] //Änderung der Proteinkonzentration von x2
        let x3 = P[2] //Änderung der Proteinkonzentration von x3

        //Differenzialgleichung die die Änderung der Proteinkonzentration von x1 beschreibt (hier die Gleichung einsetzen)
        let x1' = beta / (1. + (x3/K_d) ** n) - gamma * x1
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x2 beschreibt (hier die Gleichung einsetzen)
        let x2' = beta / (1. + (x1/K_d) ** n) - gamma * x2
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x3 beschreibt (hier die Gleichung einsetzen)
        let x3' = beta / (1. + (x2/K_d) ** n) - gamma * x3

        [|x1'; x2'; x3'|]

//Startkonzentrationen
let P0 = [|1.; 1.; 1.2|]

let Ps = 
    modelContext.OdeInt(0.,P0,dP_dt)
    |> SolPoints.take 150
    |> SolPoints.memorize 
    
let x1 = Ps |> SolPoints.toPoints 1
let x2 = Ps |> SolPoints.toPoints 2
let x3 = Ps |> SolPoints.toPoints 3

[
    Chart.Spline (x1)
    Chart.Spline (x2)
    Chart.Spline (x3)
]
|> Chart.combine
|> Chart.withSize(1100,600)
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Der gezeigte Graph zeigt wie sich die Konzentrationen der drei Proteine über die Zeit verhalten. Wir sehen die erwartet Oszillation für die angegebenen Startwerte. 

Aufgabe 4.16:<br>
Verändere die Werte der Hillkonstante. Was beobachten sie für die verschiedenen Werte von n im Bezug auf die Oszillation? Teste die folgenden Werte:
$$n=1$$
$$n=2$$
$$n=3$$
$$n=4$$
Kopiere dafür den Code Block mit den drei Differenzialgleichungen in den folgenden Code Block und ändere dann die Konstante "n".

In [ ]:
//Code hier einfügen

Hier die Antwort eingeben für die 4 Werte

Aufgabe 4.17:<br>
Verändere die Werte der Produkionskonstante $\beta$. Was beobachten sie für die verschiedenen Werte von n im Bezug auf die Oszillation? Teste die folgenden Werte:
$$\beta=1$$
$$\beta=2$$
$$\beta=3$$
$$\beta=4$$
Kopiere dafür den Code Block mit den drei Differenzialgleichungen (den originalen Code Block, nicht den mit den Veränderten Werten von der Hillkonstante) in den folgenden Code Block und ändere dann die Konstante "beta".

In [ ]:
//Code hier einfügen

Hier die Antwort eingeben für die vier Werte

Die bisherige Darstellung zeigt zwar das alle drei Proteine oszillieren, aber eine alternative Darstellung im 3D Plot, wobei die Konzentration von x1 die x-Achse, die von x2, die y-Achse, und die von x3 die z-Achse darstellen, zeigt dieses Verhalten nocheinmal anschaulicher.

In [ ]:
//Konstanten
let beta  = 5. // Konstante für die Zunahme der Proteinkonzentration
let gamma = 1.  // Konstante für die Abnahme der Proteinkonzentration 
let K_d   = 1.  // Dissoziationskonstante
let n     = 3.  // Hillkonstante

let repressilator_rhs (x1,x2,x3) t beta n =
    [|
        beta / (1. + x3 ** n) - x1
        beta / (1. + x1 ** n) - x2
        beta / (1. + x2 ** n) - x3
    |]

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->
        let x1 = P[0] //Änderung der Proteinkonzentration von x1
        let x2 = P[1] //Änderung der Proteinkonzentration von x2
        let x3 = P[2] //Änderung der Proteinkonzentration von x3

        //Differenzialgleichung die die Änderung der Proteinkonzentration von x1 beschreibt (hier die Gleichung einsetzen)
        let x1' = beta / (1. + (x3/K_d) ** n) - gamma * x1
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x2 beschreibt (hier die Gleichung einsetzen)
        let x2' = beta / (1. + (x1/K_d) ** n) - gamma * x2
        //Differenzialgleichung die die Änderung der Proteinkonzentration von x3 beschreibt (hier die Gleichung einsetzen)
        let x3' = beta / (1. + (x2/K_d) ** n) - gamma * x3

        [|x1'; x2'; x3'|]

//Startkonzentrationen
let P0 = [|1.; 1.; 1.2|]

let Ps = 
    modelContext.OdeInt(0.,P0,dP_dt)
    |> SolPoints.take 1000
    |> SolPoints.memorize 
    
let x1 = Ps |> SolPoints.toY 0
let x2 = Ps |> SolPoints.toY 1
let x3 = Ps |> SolPoints.toY 2

Chart.Line3D(x1, x2, x3)
|> Chart.withSize(1100,600)
|> Chart.withXAxisStyle("Proteinkonzentration von x1")
|> Chart.withYAxisStyle("Proteinkonzentration von x2")
|> Chart.withZAxisStyle("Proteinkonzentration von x3")

<!-- Plotly chart will be drawn inside this DIV -->